> # **Titanic - Machine Learning from Disaster**
![image.png](https://storage.googleapis.com/kaggle-competitions/kaggle/3136/logos/header.png)

Wykorzystanie uczenia maszynowego do stworzenia modelu, który przewiduje, którzy pasażerowie przeżyli katastrofę statku



**Instalowanie potrzebnych komponentów**




In [1]:
!wget https://raw.githubusercontent.com/vDeeMILv/misp/master/kaggle/titanic.zip
!unzip titanic.zip

--2020-12-28 20:24:56--  https://raw.githubusercontent.com/vDeeMILv/misp/master/kaggle/titanic.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34877 (34K) [application/zip]
Saving to: ‘titanic.zip’

titanic.zip         100%[===================>]  34.06K  --.-KB/s    in 0.003s  

2020-12-28 20:24:56 (10.6 MB/s) - ‘titanic.zip’ saved [34877/34877]

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


 **Importowanie potrzebnych biblotek**

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


**Dane treningowe**

In [3]:
training_set = pd.read_csv('train.csv')
training_set.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
training_set.isnull().sum() #sprawdzenie spójności danych 

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
training_set.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


**Dane testowe**

In [6]:
test_set = pd.read_csv('test.csv')
test_set.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
test_set.isnull().sum() #sprawdzenie spójności danych

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [8]:
test_set.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


**Uzupełnienie brakujących danych średnią**
w celu uzyskania poprawnego modelu potrzebujemy uzupełnić brakujace dane

In [9]:
training_set['Age'].fillna(training_set.Age.mean(), inplace=True)
test_set['Age'].fillna(test_set.Age.mean(), inplace=True)

**Ponowne sprawdzenie spójności danych**

In [10]:
training_set.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
test_set.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

**Utworzenie dataframe z potrzebnymi informacjami do wyznaczenia celu**

In [12]:
training_features = training_set[["Pclass", "Sex", "Age",'Parch', 'SibSp']]
training_features['Sex'] = training_features['Sex'].replace('male', 0)
training_features['Sex'] = training_features['Sex'].replace('female', 1)
training_features.head(5)

,Pclass,Sex,Age,Parch,SibSp
0,3,0,22.0,0,1
1,1,1,38.0,0,1
2,3,1,26.0,0,0
3,1,1,35.0,0,1
4,3,0,35.0,0,0


Cel i dopasowywanie

In [13]:
decision_tree_model = DecisionTreeClassifier(random_state=1)
y = training_set[["Survived"]]
train_X, val_x, train_y, val_y = train_test_split(training_features, y, random_state=1)
decision_tree_model.fit(train_X, train_y)
print("Dokładność: ",decision_tree_model.score(train_X, train_y))


Dokładność:  0.9401197604790419


In [14]:
test_features = test_set[["Pclass", "Sex", "Age",'Parch', 'SibSp']]
test_features['Sex'] = test_features['Sex'].replace('male', 0)
test_features['Sex'] = test_features['Sex'].replace('female', 1)
test_features.head(5)

,Pclass,Sex,Age,Parch,SibSp
0,3,0,34.5,0,0
1,3,1,47.0,0,1
2,2,0,62.0,0,0
3,3,0,27.0,0,0
4,3,1,22.0,1,1


In [15]:
result = decision_tree_model.predict(test_features)


In [17]:
pd.DataFrame({'PassengerId': test_set['PassengerId'], 'Survived': result,}).to_csv('results.csv', index = False)
pd.DataFrame({'PassengerId': test_set['PassengerId'], 'Survived': result,})

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


* Uzyskany wynik na keggle: 0.73923


